In [ ]:
import getpass
import os
import sys

username = getpass.getuser()
location = os.getenv("EXTERNAL_INSTANCE_URL", "")

ts_config_scheduler_root = None

# SOME PATH CONFIGURATION STUFF
if username=='neilsen' and location=="https://usdf-rsp.slac.stanford.edu":
    # Modifications of the python path with
    # sys.path do not work if the same namespace
    # is already scanned in the PYTHONPATH.
    # Get rid of the modules pre-loaded so the
    # interpreter reloads lsst.blah.blah.blah
    # after we have made our modifications
    # to sys.path.
    sys.modules.pop("lsst.ts.fbs", None)
    sys.modules.pop("lsst.ts", None)
    sys.modules.pop("lsst", None)
    
    sys.path.insert(0, "/sdf/data/rubin/shared/scheduler/packages/rubin_scheduler-3.18.1")
    sys.path.insert(0, "/sdf/data/rubin/shared/scheduler/packages/SP-2167/rubin_sim")
    sys.path.insert(0, "/sdf/data/rubin/user/neilsen/devel/rubin_nights")
    sys.path.insert(0, "/sdf/data/rubin/user/neilsen/devel/lsst_survey_sim")
    sys.path.insert(0, "/sdf/data/rubin/user/neilsen/devel/ts_fbs_utils/python")

    ts_config_scheduler_root = "/sdf/data/rubin/user/neilsen/devel/ts_config_scheduler"
    do_git_stuff = False

elif username=='lynnej' and location == "":
    ts_config_scheduler_root = "/Users/lynnej/lsst_repos/ts_config_scheduler"
    do_git_stuff = True

# SOME TOKEN CONFIGURATION STUFF
# Are you on an RSP?
if location != "":
    tokenfile = None
    site = None
# Or are you outside of an RSP? - just use USDF and your own USDF-RSP token
# See https://rsp.lsst.io/guides/auth/creating-user-tokens.html
else:
    # Substitute the location of your own tokenfile
    # If you prefer, 'get_client' will also get token info from an "ACCESS_TOKEN" environment variable
    tokenfile = os.path.join(os.path.expanduser("~"), ".lsst/usdf_rsp")
    site = 'usdf'

# Fix for Eric's tokenfile location .. we should standardize this
if username=='neilsen' and location=="https://usdf-rsp.slac.stanford.edu":
    tokenfile = "/home/n/neilsen/.lsst/usdf_access_token"

# For everyone who did not set ts_config_scheduler_root path
if ts_config_scheduler_root is None:
    # Just make a new clone for ts_config_scheduler
    ts_config_scheduler_root = "."
    do_git_stuff = True


assert isinstance(ts_config_scheduler_root, str), "Please set ts_config_scheduler_root"

In [ ]:
import getpass
import os
import warnings
import copy
import logging

import numpy as np
import pandas as pd
import sqlite3
import healpy as hp
import matplotlib.pyplot as plt
import colorcet as cc
import skyproj
from IPython.display import display, HTML

from astropy.time import Time, TimeDelta
from astropy.coordinates import SkyCoord
import astropy.units as u

from rubin_scheduler.scheduler.utils import SchemaConverter
from rubin_scheduler.site_models import Almanac
from rubin_scheduler.scheduler.features import Conditions
from rubin_scheduler.utils import ddf_locations, angular_separation, approx_ra_dec2_alt_az, Site, SURVEY_START_MJD

import rubin_sim.maf as maf
from rubin_sim.data import get_baseline

from rubin_nights import connections
import rubin_nights.lfa_data as rn_lfa
import rubin_nights.dayobs_utils as rn_dayobs
import rubin_nights.plot_utils as rn_plots
import rubin_nights.augment_visits as augment_visits
import rubin_nights.rubin_scheduler_addons as rn_sch
import rubin_nights.rubin_sim_addons as rn_sim
import rubin_nights.observatory_status as observatory_status
import rubin_nights.scriptqueue as scriptqueue
import rubin_nights.scriptqueue_formatting as scriptqueue_formatting
import rubin_nights.targets_and_visits as targets_and_visits

import importlib

from lsst_survey_sim import lsst_support, simulate_lsst, plot

band_colors = rn_plots.PlotStyles.band_colors
logging.getLogger('lsst_survey_sim').setLevel(logging.INFO)

#%load_ext memory_profiler

In [ ]:
day_obs = rn_dayobs.day_obs_str_to_int(rn_dayobs.today_day_obs())

survey_start = SURVEY_START_MJD
programs = ["BLOCK-365", "BLOCK-407", "BLOCK-408"]

sunset, sunrise = rn_dayobs.day_obs_sunset_sunrise(day_obs, sun_alt=-12)
print(f"DayObs {day_obs}, -12 deg sunset {sunset.iso}, -12 deg sunrise {sunrise.iso}")

In [ ]:
endpoints = connections.get_clients(tokenfile)

In [ ]:
refresh_visits = True

day_obs_min = 20251026
day_obs_max = rn_dayobs.day_obs_str_to_int(rn_dayobs.today_day_obs())

if refresh_visits:
    skip_imgtypes = ["bias", "flat", "dark"]
    query = ( 
        "select *, q.* from cdb_lsstcam.visit1 left join cdb_lsstcam.visit1_quicklook as q on visit1.visit_id = q.visit_id "
        f"where visit1.day_obs >= {day_obs_min} and visit1.day_obs <= {day_obs_max} and img_type != 'bias' and img_type != 'flat' and img_type != 'dark'"
          )

    # query = ( 
    #     "select *, q.* from cdb_lsstcam.visit1 left join cdb_lsstcam.visit1_quicklook as q on visit1.visit_id = q.visit_id "
    #     "where visit1.day_obs >= 20250701 and visit1.day_obs <= 20250709 and img_type != 'bias' and img_type != 'flat' and img_type != 'dark'"
    #       )
    visits = endpoints['consdb'].query(query)
    visits = augment_visits.augment_visits(visits, "lsstcam")  
    visits.reset_index(inplace=True)
    visits.drop("index", axis=1, inplace=True)
    with warnings.catch_warnings(record=True) as w:
        warnings.simplefilter("always") 
        visits.to_hdf('v_now.h5', key='visits')
else:
    visits = pd.read_hdf('v_now.h5')
print("all visits:", len(visits), "science visits:", len(visits.query("science_program in @programs")))


# Flag program changes 
program_change = np.where((visits.science_program[:-1].values != visits.science_program[1:].values))[0]
program_change = program_change + 1
pmask = np.zeros(len(visits))
pmask[0] = 1
pmask[program_change] = 1
visits["program_change"] = pmask

# Flag filter changes 
filter_change = np.where((visits.band[:-1].values != visits.band[1:].values) 
                         & (visits.day_obs[:-1].values == visits.day_obs[1:].values))[0]
filter_change = filter_change + 1
fmask = np.zeros(len(visits))
fmask[filter_change] = 1
visits["filter_change"] = fmask

# calculate slew times 
wait_before_slew = 1.6
settle = 1.5
max_scatter = 5
visits, slewing = rn_sch.add_model_slew_times(visits, endpoints['efd'], model_settle=wait_before_slew + settle, dome_crawl=False, slew_while_changing_filter=False)
valid_overhead = np.min([np.where(np.isnan(visits.slew_model.values), 0, visits.slew_model.values) + max_scatter, visits.visit_gap.values], axis=0)
visits["overhead"] = valid_overhead

# Need to remove faults for the first visit of the night or where there was a different program we didn't fetch
# (could skip *some* of this by fetching all visits, but might still have some missing due to flats?)
skipped_visits = np.concatenate([np.array([0]), np.where(visits.visit_id[:-1].values + 1 != visits.visit_id[1:].values)[0] + 1])

fault = visits.visit_gap - valid_overhead
fault[skipped_visits] = np.nan
visits["fault_idle"] = fault

visits.loc[skipped_visits, 'model_gap'] = np.nan

bad_visit_ids = augment_visits.fetch_excluded_visits("lsstcam")
visits['bad_flag'] = np.zeros(len(visits), int)
idx = visits.query("visit_id in @bad_visit_ids").index
visits.loc[idx, 'bad_flag'] = 1

sci = visits.query("science_program in @programs")

## thoughts -- need to mark time associated with bad visits as fault somehow .. 

In [ ]:
cv = sci.groupby(["day_obs", "band"]).agg({"obs_start_mjd": "count"}).reset_index("band")
cols = [c for c in ['u', 'g', 'r', 'i', 'z', 'y'] if c in cv.band.values]
cv = cv.pivot(columns="band").droplevel(0, axis=1)[cols]
cv['all'] = cv.sum(axis=1)
display(cv)
ov = sci.groupby("observation_reason").agg({"obs_start_mjd": "count", "target_name": "unique"}).rename({"obs_start_mjd": "count"}, axis=1)
def split_regions(x):
    regions = set()
    for k in x.target_name:
        regions = regions.union(set([kk.replace(' ', '') for kk in k.split(',')]))
    regions = list(regions)
    regions.sort()
    return regions
ov.target_name = ov.apply(split_regions, axis=1)
display(HTML(ov.to_html()))

In [ ]:
#q = visits
#tv, cols, to, nv, v = targets_and_visits.targets_and_visits(Time(q.obs_start_mjd.min() - 0.1/24, format='mjd', scale='tai'), Time(q.obs_start_mjd.max() + 0.1/24, format='mjd', scale='tai'), endpoints)

In [ ]:
# Typical times required for some blocks ..
dd = []
q = visits
for ddayobs in q.day_obs.unique():
    qq = visits.query("day_obs == @ddayobs")
    p_start = qq.query("program_change == 1")
    t = list(zip(p_start.science_program.values, np.diff(p_start.obs_start_mjd.values) * 24 * 60))
    a = pd.DataFrame([tti[0] for tti in t], columns=['block'])
    b = pd.DataFrame([tti[1] for tti in t], columns=['time'])
    c = pd.DataFrame([ddayobs]*len(a), columns=['dayobs'])
    qq = a.join(b).join(c)
    dd.append(qq)

dd = pd.concat(dd)
dd.groupby("block").agg({'time': ('mean', 'median', 'max', 'min', 'count'), 'dayobs': ('min', 'max')}, axis=1)

In [ ]:
dayobs = visits.day_obs.unique()
dayobs = visits.query("day_obs == 20251108").day_obs.unique()
q = visits.query("science_program in @programs and day_obs in @dayobs")
total_time = (q.shut_time.sum() + q.overhead.sum() + q.fault_idle.sum())/3600
total_onsky = q.exp_time.sum()/3600
total_req = (q.shut_time.sum() + q.overhead.sum())/3600
total_fault_idle = q.fault_idle.sum()/3600
dd = pd.DataFrame([total_time, total_onsky, total_req, total_fault_idle, len(q), len(q)*30/3600], index=["time", "onsky", "req", "fault_idle", "nvis", 'estimate time onsky'], columns=["all " + "_".join(programs)])
display(dd.T)
cols = ['visit_id', 'img_type', 'observation_reason', 'obs_start_mjd', 'band', 's_ra', 's_dec', 'sky_rotation', 'altitude', 'azimuth', 'physical_rotator_angle',
                           'clouds', 'fwhm_eff', 'filter_change', 'slew_distance', 'slew_model', 'visit_gap', 'model_gap',] #, 'overhead', 'fault_idle']
#display(sci.query("model_gap > 2 and model_gap < 10")[cols])

print(f"Min/median/mean overheads: {q.overhead.min():.2f} {q.overhead.median():.2f} {q.overhead.mean():.2f}")
print(f"Min/median/mean visit gaps: {q.visit_gap.min():.2f} {q.visit_gap.median():.2f} {q.visit_gap.mean():.2f}")

#q = visits.query("science_program in @programs")
fig, axes = plt.subplots(1, 3, figsize=(20, 5))
ax = axes[0]
ax.plot(q.visit_gap, q.slew_model, 'k.')
x = np.arange(0, 500, 1)
ax.plot(x, x, alpha=0.3)
ax.fill_betweenx(x1=x+max_scatter, x2=x, y=x, color='pink', alpha=0.2) 
ax.set_xlim(0, 30)
ax.set_ylim(0, 30)
ax.grid(alpha=0.4)
ax.set_xlabel("Visit gap (seconds)")
ax.set_ylabel("Slew model (seconds)")

ax = axes[1]
ax.plot(q.visit_gap, q.slew_model, 'k.')
x = np.arange(0, 500, 1)
ax.plot(x, x, alpha=0.3)
ax.fill_betweenx(x1=x+max_scatter, x2=x, y=x, color='pink', alpha=0.2) 
#ax.set_xlim(0, 30)
#ax.set_ylim(0, 30)
ax.grid(alpha=0.4)
ax.set_xlabel("Visit gap (seconds)")
ax.set_ylabel("Slew model (seconds)")

ax = axes[2]
ax.plot(q.slew_distance, q.model_gap, 'k.')
ax.set_ylim(-10, 10)
ax.grid(alpha=0.3)
ax.set_xlabel("Slew distance (deg)")
_ = ax.set_ylabel("visit_gap - slew_model (seconds)")

In [ ]:
dayobs = visits.day_obs.unique()
#dayobs = [20251108]
q = visits.query("science_program in @programs and day_obs in @dayobs")
# Calculate "open shutter fraction" without faults
slew_eff = (q.exp_time.sum()) / (q.dark_time.sum() + q.overhead.sum())
ideal_eff = (q.exp_time.sum()) / (q.dark_time.sum() + q.slew_model_ideal.sum())
print(f"Slew efficiency factor: {slew_eff: 0.2f}")
print(f"Ideal model efficiency equivalent: {ideal_eff: 0.2f}")
print(f"Ratio - slew / ideal {slew_eff / ideal_eff :0.2f}")

In [ ]:
# Get ticketed time lost
time_lost_logs = endpoints['narrative_log'].query_log(rn_dayobs.day_obs_to_time(visits.day_obs.min()), rn_dayobs.day_obs_to_time(visits.day_obs.max()), 
                                                     {"min_time_lost": "0.00001"})
time_lost_logs = time_lost_logs.query("not component.str.contains('AuxTel')")
def time_to_day_obs(x):
    return int(x.date_begin.split("T")[0].replace("-", ""))
time_lost_logs['day_obs'] = time_lost_logs.apply(time_to_day_obs, axis=1)
log_fault = time_lost_logs.query("time_lost_type == 'fault'").groupby('day_obs').agg({'time_lost': 'sum'}).rename({"time_lost": "log_fault"}, axis=1)
log_weather = time_lost_logs.query("time_lost_type == 'weather'").groupby('day_obs').agg({'time_lost': 'sum'}).rename({"time_lost": "log_weather"}, axis=1)
log_lost = log_fault.merge(log_weather, how="outer", on="day_obs")

In [ ]:
# time from -12 degree twilight to first science visit, and time from last visit to -12 deg
# and fault times (total)
dd = {}
for ddayobs in visits.day_obs.unique():
    q = visits.query("day_obs == @ddayobs")
    nvis = len(q)
    all_fault_idle = round(q.fault_idle.sum()/60/60, 2)
    gap_fault_idle = round(q.query("visit_gap > 5*60").fault_idle.sum()/60/60, 2)
    sunset, sunrise = rn_dayobs.day_obs_sunset_sunrise(int(ddayobs), sun_alt=-12)
    qq = q.query("science_program in @programs")
    if len(qq) == 0:
        twi_to_start = (sunrise.mjd - sunset.mjd) * 24 - gap_fault_idle
        end_to_twi = 0
    else:
        twi_to_start = (qq.obs_start_mjd.min() - sunset.mjd) * 24
        end_to_twi = (sunrise.mjd - qq.obs_end_mjd.max()) * 24
    night_hours = (sunrise.mjd - sunset.mjd) * 24
    fbs = q.query("science_program in @programs")
    time_in_fbs = (fbs.obs_end_mjd.max() - fbs.obs_start_mjd.min()) * 24 
    time_predict_in_fbs = (fbs.dark_time.sum() + fbs.slew_model_ideal.sum()) / 60 / 60        
    dd[ddayobs] = [ddayobs, night_hours, nvis, twi_to_start, end_to_twi,  time_in_fbs, time_predict_in_fbs,  all_fault_idle, gap_fault_idle,]
dd = pd.DataFrame(dd, index=["day_obs", "night_hours", "n_visits", 
                              "twi_to_start", "end_to_twi", 
                             "time_in_fbs", "time_predict_in_fbs", 
                             "total_fault_idle", "total_fault_idle_gap" ]).T
dd = dd.merge(ticketed, how="outer", on="day_obs")
dd['day_obs'] = dd['day_obs'].astype(int)
dd.set_index("day_obs", inplace=True)
missing_night_ends = np.min([dd.night_hours.values - dd.twi_to_start.values - dd.end_to_twi.values, np.ones(len(dd))*1.8], axis=0)
ratio_night_used = (dd.night_hours - missing_night_ends) / dd.night_hours
ratio = round((dd.night_hours - missing_night_ends - dd.total_fault_idle_gap) / (dd.night_hours), 2)
ratio = np.where(ratio <= 0, 0, ratio)
dd['ratio_all_times'] = ratio * slew_eff / ideal_eff
dd['ratio_fbs_times'] = round(dd.time_predict_in_fbs / dd.time_in_fbs * ratio_night_used, 2)
dd['ratio_sim_ref'] = round((dd.night_hours - 0.37) / dd.night_hours, 2)

non_ratio_cols = [c for c in dd if 'ratio' not in c]
dd.loc['total', non_ratio_cols] = dd[non_ratio_cols].sum(axis=0)
ratio_cols = [c for c in dd if 'ratio' in c]
dd.loc['total', ratio_cols] = dd[ratio_cols].mean(axis=0)
display(dd)

print(f"fault+idle (hours) - total: {dd.total_fault_idle_gap.sum():.2f} mean: {np.nanmean(dd.total_fault_idle_gap):.2f}")

# make an average ratio, with a fudge factor for time lost at ends of night
ave_ratio = np.nanmean((dd.night_hours - dd.total_fault_idle_gap - 1.8)/dd.night_hours)
ave_sim_ratio = np.mean((dd.night_hours - 0.37)/(dd.night_hours))
print(f"some kind of average ratio {ave_ratio:.2f} compare to {ave_sim_ratio:.2f}")
print(f"Slew performance ratio {slew_eff / ideal_eff:.2f}")
print(f"System availability estimate all: {np.nanmean(dd.ratio_all_times):.2f}")
print(f"System availability estimate fbs: {np.nanmean(dd.ratio_fbs_times):.2f}")

In [ ]:
x = np.arange(0, len(dd))
y = ((dd.night_hours - dd.total_fault_idle_gap - 1.8)/dd.night_hours) / ((dd.night_hours - 0.37)/(dd.night_hours)) * slew_eff / ideal_eff
plt.plot(x, y, marker='.')
_ = plt.xticks(x, dd.index, rotation=90)
_ = plt.ylabel("efficiency relative to v5.1")
plt.figure()
_ = plt.hist(y, bins=20)

In [ ]:
bins = np.arange(0.5, 2.8, 0.05)
q = visits.query("science_program == 'BLOCK-407' and day_obs >= 20251102")
_ = plt.hist(q.fwhm_eff.values, bins=bins, density=False, alpha=0.4, label='407')
q = visits.query("science_program == 'BLOCK-408' and day_obs >= 20251102")
_ = plt.hist(q.fwhm_eff.values, bins=bins, density=False, alpha=0.4, label='408')
plt.legend()
plt.ylabel("Fraction of images")
plt.xlabel("FWHM (arcseconds)")

In [ ]:
dayobs = [20251116]
q = visits.query("science_program in @programs and day_obs in @dayobs")
qq = q.query("slew_distance < 0.001")
plt.figure(figsize=(20, 5))
plt.plot(q.seq_num, q.altitude, 'r.')
plt.plot(qq.seq_num, qq.altitude, 'o', markerfacecolor='None', markeredgecolor='b')
plt.xlabel("seq_num")
plt.ylabel("Altitude (deg)")
plt.figure(figsize=(20,  5))
plt.plot(q.seq_num, q.physical_rotator_angle, 'r.')
plt.plot(qq.seq_num, qq.physical_rotator_angle, 'o', markerfacecolor='None', markeredgecolor='b')
plt.xlabel("seq_num")
plt.ylabel("physical rotator angle (deg)")


In [ ]:
compare_baseline  = False
if compare_baseline:
    try:
        opsdb = "/Users/lynnej/opsim/fbs_5.1/baseline_v5.1.0_10yrs.db"
        conn = sqlite3.connect(opsdb)
    except: 
        opsdb = get_baseline()
        conn = sqlite3.connect(opsdb)
    
    b_visits = pd.read_sql(f"select * from observations where night < 365", conn)
    
    tconsdb = sci.obs_start_mjd.max() - sci.obs_start_mjd.min()
    print(f"comparing visits within {tconsdb} days of start")
    baseline_visits = b_visits.query("observationStartMJD - observationStartMJD.min() < @tconsdb")
    
    xx = baseline_visits.groupby("observation_reason").agg({'observationStartMJD': "count"}).rename({"observationStartMJD": "baseline_v5.1.0"}, axis=1)
    xa = sci.groupby("observation_reason").agg({'obs_start_mjd': 'count'}).rename({"obs_start_mjd": "consdb"}, axis=1)
    print(len(sci), len(baseline_visits), 'sci/baseline', len(sci)/len(baseline_visits))
    x = pd.concat([xa, xx], axis=1)
    x.loc['total'] = x.sum(axis=0)
    display(x)

In [ ]:
ddf_visits = visits.query("bad_flag==0 and observation_reason.str.contains('DD') or observation_reason.str.contains('ddf')").copy()
if len(ddf_visits) > 0:
    print("n visits per band")
    ddf_visits.loc[:, 'observation_reason'] = ddf_visits.observation_reason.str.lower()
    ss = ddf_visits.groupby(["observation_reason", "band"]).agg({'obs_start_mjd': 'count'})
    ss = ss.reset_index('band').pivot(columns=["band"]).droplevel(0, axis=1)
    ss['all'] = ss.sum(axis=1)
    order = ['u', 'g', 'r', 'i', 'z', 'y', 'all']    
    display(ss.query("observation_reason.str.contains('dd')")[[o for o in order if o in ss.columns]])
    
    print("n days per band")
    ss = ddf_visits.groupby(["observation_reason", "band"]).agg({'day_obs': 'nunique'})
    ss = ss.reset_index('band').pivot(columns=["band"]).droplevel(0, axis=1)
    ss['all'] = ddf_visits.groupby("observation_reason").agg({'day_obs': 'nunique'})
    display(ss.query("observation_reason.str.contains('dd')")[[o for o in order if o in ss]])

In [ ]:
run_calc = True
if run_calc:
    gsci = sci.query("bad_flag == 0")
    nvisits = {}
    coadd = {}
    m_nvis = maf.CountMetric(col='obs_start_mjd', metric_name = "Nvisits")
    m_coadd = maf.Coaddm5Metric(m5_col='cat_m5')
    s = maf.HealpixSlicer(nside=64, lon_col='s_ra', lat_col='s_dec', rot_sky_pos_col_name = 'sky_rotation')
    for b in ['u', 'g', 'r', 'i', 'z', 'y', 'all']:
        constraint = f"{b}"
        if b == 'all':
            opsvis = gsci.to_records()
        else:
            opsvis = gsci.query("band == @b").to_records()
        nvisits[b] = maf.MetricBundle(m_nvis, s, constraint)
        coadd[b] = maf.MetricBundle(m_coadd, s, constraint)
        g = maf.MetricBundleGroup({f'nvisits {b}': nvisits[b], f'coadd {b}': coadd[b]}, None)
        if len(opsvis) > 0:
            g.run_current(constraint, opsvis)            

In [ ]:
background = plot.get_background(nside=64)

fig, ax = plt.subplots(nrows=2, ncols=3, figsize=(16, 10),)
axdict = {"u": ax[0][0], "g": ax[0][1], "r": ax[0][2],
          "i": ax[1][0], "z": ax[1][1], "y": ax[1][2], "all": None}
for b in ["u", "g", "r", "i", "z", "y"]:
    if nvisits[b].metric_values is not None:
        if len(nvisits[b].metric_values.compressed()) > 1:
            vmax = np.percentile(nvisits[b].metric_values.compressed(), 95)
        else:
            vmax = None
        label_dec = False
        if b == 'u' or b == 'i':
            label_dec = True
        fig = plot.make_plot(nvisits[b], background=background, proj='McBryde', vmax=vmax, ax=axdict[b], title=f"LSSTCam band {b}", label_dec=label_dec)
fig.tight_layout()
#fig.savefig(os.path.join(out_dir, f"lsstcam_nvisits_band.png"), bbox_inches='tight')

vmax = np.percentile(nvisits['all'].metric_values.compressed(), 95)
fig = plot.make_plot(nvisits['all'], background=background, proj='mcbryde', vmin=None, vmax=vmax, ax=None, title=f"LSSTCam visits")
fig = plot.make_plot(nvisits['all'], background=background, proj='laea', vmin=None, vmax=vmax, ax=None, title=f"LSSTCam visits")
#fig.savefig(os.path.join(out_dir, f"lsstcam_nvisits.png"), bbox_inches='tight')